In [169]:
import numpy as np
import pandas as pd
import datetime

### Import and Feature Selection

In [45]:
df = pd.read_csv('original.csv', low_memory=False)

In [46]:
df.shape

(4508586, 18)

In [47]:
df = df.reset_index()

In [37]:
df.columns.to_list()

['index',
 'cmte_id',
 'cand_id',
 'cand_nm',
 'contbr_nm',
 'contbr_city',
 'contbr_st',
 'contbr_zip',
 'contbr_employer',
 'contbr_occupation',
 'contb_receipt_amt',
 'contb_receipt_dt',
 'receipt_desc',
 'memo_cd',
 'memo_text',
 'form_tp',
 'file_num',
 'tran_id',
 'election_tp']

In [48]:
df = df.drop(['index', 'cmte_id', 'file_num', 'contb_receipt_dt', 'receipt_desc', \
                        'memo_cd', 'memo_text', 'form_tp', 'tran_id', 'election_tp'],axis=1)

In [49]:
df.head()

,cand_id,cand_nm,contbr_nm,contbr_city,contbr_st,contbr_zip,contbr_employer,contbr_occupation,contb_receipt_amt
0,"Sanford, Marshall","KEITHLEY, BRAD",ANCHORAGE,AK,99501,ALASKANS FOR SUSTAINABLE BUDGETS,MANAGING DIRECTOR,1000.0,09-SEP-19
1,"Sanford, Marshall","ANDERSON, FRANKLIN",CHICKASAW,AL,366111850,RETIRED,RETIRED,100.0,28-SEP-19
2,"Sanford, Marshall","HIXON, SARAH",BANKS,AL,360054123,RETIRED,RETIRED,500.0,10-SEP-19
3,"Sanford, Marshall","FOLMAR, DAMON",COLUMBIANA,AL,35051,SELF EMPLOYED,SALES/MARKETING,250.0,08-SEP-19
4,"Sanford, Marshall","ANDERSON, FRANKLIN",CHICKASAW,AL,366111850,RETIRED,RETIRED,250.0,10-SEP-19


In [50]:
df.columns.to_list()

['cand_id',
 'cand_nm',
 'contbr_nm',
 'contbr_city',
 'contbr_st',
 'contbr_zip',
 'contbr_employer',
 'contbr_occupation',
 'contb_receipt_amt']

In [51]:
df.columns=['candidate',
 'donor',
 'city',
 'state',
 'zip',
 'employer',
 'occupation',
 'amount',
 'date']

In [52]:
df.head()

,candidate,donor,city,state,zip,employer,occupation,amount,date
0,"Sanford, Marshall","KEITHLEY, BRAD",ANCHORAGE,AK,99501,ALASKANS FOR SUSTAINABLE BUDGETS,MANAGING DIRECTOR,1000.0,09-SEP-19
1,"Sanford, Marshall","ANDERSON, FRANKLIN",CHICKASAW,AL,366111850,RETIRED,RETIRED,100.0,28-SEP-19
2,"Sanford, Marshall","HIXON, SARAH",BANKS,AL,360054123,RETIRED,RETIRED,500.0,10-SEP-19
3,"Sanford, Marshall","FOLMAR, DAMON",COLUMBIANA,AL,35051,SELF EMPLOYED,SALES/MARKETING,250.0,08-SEP-19
4,"Sanford, Marshall","ANDERSON, FRANKLIN",CHICKASAW,AL,366111850,RETIRED,RETIRED,250.0,10-SEP-19


In [53]:
df.candidate = df.candidate.replace('Sanders, Bernard', "Bernie Sanders")
df.candidate = df.candidate.replace('Buttigieg, Pete', "Pete Buttigieg")

In [70]:
df2 = df[(df.candidate  == 'Pete Buttigieg')|(df.candidate == 'Bernie Sanders')]

In [71]:
df2.shape

(1324765, 9)

In [72]:
df2.head()

,candidate,donor,city,state,zip,employer,occupation,amount,date
2817626,Bernie Sanders,"SUNDT, DICK",OWENSVILLE,AA,45160,NOT EMPLOYED,NOT EMPLOYED,100.0,27-MAR-19
2817627,Bernie Sanders,"SMITH, BRANDON",HELSINKI,AA,00160,FAST ENTERPRISES,IMPLEMENTATION CONSULTANT,1700.0,20-FEB-19
2817628,Bernie Sanders,"SMITH, BRANDON",HELSINKI,AA,00160,FAST ENTERPRISES,IMPLEMENTATION CONSULTANT,1000.0,20-FEB-19
2817629,Bernie Sanders,"ZAZZARINO, LISA",APO,AE,090120074,NOT EMPLOYED,NOT EMPLOYED,25.0,15-NOV-19
2817630,Bernie Sanders,"ZAZZARINO, LISA",APO,AE,090120074,NOT EMPLOYED,NOT EMPLOYED,25.0,15-DEC-19


In [74]:
df2['zip_code'] = df['zip'].astype(str).str[0:5]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [75]:
df2['zip_code'] = df2['zip_code'].replace(',','')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [76]:
df2 = df2.drop(['zip','city','employer','occupation'], axis=1)

In [186]:
df2.shape

(1324765, 6)

### Focus on Obama-Trump States

In [187]:
mask = (df2.state == 'FL') | (df2.state == 'IA') | (df2.state == 'MI') |\
(df2.state == 'OH') | (df2.state == 'PA') | (df2.state == 'WI')

In [188]:
df3 = df2.loc[mask]

In [189]:
df3.shape

(191235, 6)

### Clean Dates

In [190]:
date_ = df3['date'].apply(lambda x: pd.Series(str(x).split('-')))

In [191]:
date_.columns = ['day','month','year']

In [192]:
date_.month = date_.month.replace('JAN', '01')
date_.month = date_.month.replace('FEB', '02')
date_.month = date_.month.replace('MAR', '03')
date_.month = date_.month.replace('APR', '04')
date_.month = date_.month.replace('MAY', '05')
date_.month = date_.month.replace('JUN', '06')
date_.month = date_.month.replace('JUL', '07')
date_.month = date_.month.replace('AUG', '08')
date_.month = date_.month.replace('SEP', '09')
date_.month = date_.month.replace('OCT', '10')
date_.month = date_.month.replace('NOV', '11')
date_.month = date_.month.replace('DEC', '12')

In [193]:
date_.year = date_.year.replace('19', '2019')
date_.year = date_.year.replace('18', '2018')
date_.year = date_.year.replace('17', '2017')
date_.year = date_.year.replace('16', '2016')

In [194]:
date_2 = date_

In [195]:
date_ = pd.DataFrame(pd.to_datetime(date_, errors='raise', dayfirst=True))

In [196]:
date_.columns=['date_clean']

In [197]:
date_.head()

,date_clean
3107298,2019-12-31
3107299,2019-12-31
3107300,2019-12-31
3107301,2019-12-31
3107302,2019-12-31


In [199]:
date_2 = date_2.drop(['day', 'year'], axis=1)

In [200]:
dates = pd.merge(date_, date_2, how='left', left_index=True, right_index=True)

In [201]:
df4 = pd.merge(df3, dates, how='left', left_index=True, right_index=True)

In [202]:
df4.head()

,candidate,donor,state,amount,date,zip_code,date_clean,month
3107298,Bernie Sanders,"YARNELL, RICHARD",FL,1.0,31-DEC-19,34787,2019-12-31,12
3107299,Bernie Sanders,"YARNELL, RICHARD",FL,1.0,31-DEC-19,34787,2019-12-31,12
3107300,Bernie Sanders,"YARNELL, RICHARD",FL,1.0,31-DEC-19,34787,2019-12-31,12
3107301,Bernie Sanders,"YARNELL, RICHARD",FL,1.0,31-DEC-19,34787,2019-12-31,12
3107302,Bernie Sanders,"YARNELL, RICHARD",FL,1.0,31-DEC-19,34787,2019-12-31,12


### Limit to 2019

In [203]:
mask = (df4['date_clean'] > '2018/12/31')

In [204]:
df4.loc[mask]

,candidate,donor,state,amount,date,zip_code,date_clean,month
3107298,Bernie Sanders,"YARNELL, RICHARD",FL,1.00,31-DEC-19,34787,2019-12-31,12
3107299,Bernie Sanders,"YARNELL, RICHARD",FL,1.00,31-DEC-19,34787,2019-12-31,12
3107300,Bernie Sanders,"YARNELL, RICHARD",FL,1.00,31-DEC-19,34787,2019-12-31,12
3107301,Bernie Sanders,"YARNELL, RICHARD",FL,1.00,31-DEC-19,34787,2019-12-31,12
3107302,Bernie Sanders,"YARNELL, RICHARD",FL,1.00,31-DEC-19,34787,2019-12-31,12
...,...,...,...,...,...,...,...,...
4140934,Pete Buttigieg,"WOOD, MARGE",WI,20.20,06-OCT-19,53711,2019-10-06,10
4140935,Pete Buttigieg,"WOOD, MARGE",WI,25.00,08-OCT-19,53711,2019-10-08,10
4140936,Pete Buttigieg,"WOOD, MARGE",WI,77.02,12-OCT-19,53711,2019-10-12,10
4140937,Pete Buttigieg,"WOOD, MARGE",WI,25.00,11-OCT-19,53711,2019-10-11,10


In [205]:
df4 = df4.loc[mask]

In [206]:
df4 = df4.drop(['date'], axis=1)

In [207]:
df4.head()

,candidate,donor,state,amount,zip_code,date_clean,month
3107298,Bernie Sanders,"YARNELL, RICHARD",FL,1.0,34787,2019-12-31,12
3107299,Bernie Sanders,"YARNELL, RICHARD",FL,1.0,34787,2019-12-31,12
3107300,Bernie Sanders,"YARNELL, RICHARD",FL,1.0,34787,2019-12-31,12
3107301,Bernie Sanders,"YARNELL, RICHARD",FL,1.0,34787,2019-12-31,12
3107302,Bernie Sanders,"YARNELL, RICHARD",FL,1.0,34787,2019-12-31,12


### Clean Donors

In [208]:
donor_ = df4['donor'].apply(lambda x: pd.Series(str(x).split(',')))

In [209]:
donor_.head()

,0,1
3107298,YARNELL,RICHARD
3107299,YARNELL,RICHARD
3107300,YARNELL,RICHARD
3107301,YARNELL,RICHARD
3107302,YARNELL,RICHARD


In [210]:
donor_.columns = ['last_name', 'first_name']

In [134]:
np.sum(donor_.isnull())

last_name     0
first_name    0
dtype: int64

In [211]:
donor_['first_name'] = donor_['first_name'].str.strip()
donor_['first_name'] = donor_['first_name'].str.replace(' MS.$','')
donor_['first_name'] = donor_['first_name'].str.replace(' MRS.$','')
donor_['first_name'] = donor_['first_name'].str.replace(' MR.$','')
donor_['first_name'] = donor_['first_name'].str.replace(' JR.$','')
donor_['first_name'] = donor_['first_name'].str.replace(' SR.$','')
donor_['first_name'] = donor_['first_name'].str.replace(' DR.$','')
donor_['first_name'] = donor_['first_name'].str.replace('III','')
donor_['first_name'] = donor_['first_name'].str.title()

donor_['last_name'] = donor_['last_name'].str.strip()
donor_['last_name'] = donor_['last_name'].str.replace('.','')
donor_['last_name'] = donor_['last_name'].str.title()

In [146]:
donor_.sample(30)

,last_name,first_name
3779470,Pietrowiak,Mike
3132585,Moreno,Brian
3532023,Battisti,Sean
3901563,Lobovits,Michelle
4075276,Pankratz,David
3614270,Scrimenti,Michele
3777627,Wester,Gerald
4079107,Alderfer,Rick
4137025,Rush,Robert
3132848,Mendoza,Michael


In [147]:
donor_.head()

,last_name,first_name
3107298,Yarnell,Richard
3107299,Yarnell,Richard
3107300,Yarnell,Richard
3107301,Yarnell,Richard
3107302,Yarnell,Richard


In [212]:
donor_['full_name'] = donor_['first_name'] + ' ' + donor_['last_name']

In [213]:
donor_ = donor_.drop(['last_name', 'first_name'], axis=1)

In [214]:
donor_.head()

,full_name
3107298,Richard Yarnell
3107299,Richard Yarnell
3107300,Richard Yarnell
3107301,Richard Yarnell
3107302,Richard Yarnell


In [215]:
df5 = pd.merge(df4, donor_, how='left', left_index=True, right_index=True)

In [216]:
df5 = df5.drop(['donor'], axis=1)

In [217]:
df5.columns.to_list()

['candidate',
 'state',
 'amount',
 'zip_code',
 'date_clean',
 'month',
 'full_name']

In [218]:
df5.columns = [['candidate', 'state', 'amount', 'zip', 'date', 'month', 'donor']]

In [219]:
df5.head()

,candidate,state,amount,zip,date,month,donor
3107298,Bernie Sanders,FL,1.0,34787,2019-12-31,12,Richard Yarnell
3107299,Bernie Sanders,FL,1.0,34787,2019-12-31,12,Richard Yarnell
3107300,Bernie Sanders,FL,1.0,34787,2019-12-31,12,Richard Yarnell
3107301,Bernie Sanders,FL,1.0,34787,2019-12-31,12,Richard Yarnell
3107302,Bernie Sanders,FL,1.0,34787,2019-12-31,12,Richard Yarnell


### Save to csv

In [228]:
df5.to_csv('pete_bernie.csv')